In [ ]:
import os
from pymongo import UpdateOne
from tqdm import tqdm
from pymongo import MongoClient
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import os
import time
import random
client = MongoClient("localhost", 29012)
db = client["test-database"]
collection_json = db["collection-json"]

In [ ]:
for item in tqdm(list(collection_json.find({'Country':"US", 'OCR': {'$exists': True}, 'Title': {'$exists': True}, 'C_Application Date': {'$exists': True}, 'C_Publication Date': {'$exists': True}, 'clean_applicants': {'$exists': True}, 'clean_inventor': {'$exists': True}}).limit(100))):
    patent_number = f"{item['Country']}{item['Publication Number']}{item['Doc_kind']}"
    response = requests.get(f'https://patents.google.com/patent/{patent_number}/en?oq={patent_number}')
    soup = BeautifulSoup(response.text, 'html.parser')
    pdf_urls = [a['href'] for a in soup.find_all('a', href=True) if a['href'].startswith('https://patentimages.storage.googleapis.com') and a['href'].endswith('.pdf')]
    print(pdf_urls)
    # Download the PDF from the URL
    pdf_url = pdf_urls[0]
    folder_name = 'us_sample_patents'
    os.makedirs(folder_name, exist_ok=True)
    pdf_file_path = os.path.join(folder_name, f"{patent_number}.pdf")

    response = requests.get(pdf_url)
    with open(pdf_file_path, "wb") as pdf_file:
        pdf_file.write(response.content)

    print(f"PDF downloaded and saved to {pdf_file_path}")
    time.sleep(random.uniform(2, 7))
